In [1]:
from ibl_pipeline.analyses import behavior
from ibl_pipeline import behavior as behavior_ingest
from ibl_pipeline import subject, action, acquisition
from ibl_pipeline.utils import psychofit as psy
import plotting_utils as putils
from uuid import UUID
import matplotlib.pyplot as plt
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

Connecting shan@datajoint.internationalbrainlab.org:3306
Connected to https://alyx.internationalbrainlab.org as vathes


In [2]:
subject.Subject & 'subject_nickname="KS004"'

subject_uuid,subject_nickname nickname,sex sex,subject_birth_date birth date,ear_mark ear mark,subject_line name,subject_source name of source,protocol_number protocol number,subject_description,subject_ts
05e97eea-64e2-4e52-a78a-5b78875b4fa3,KS004,F,2018-12-04,None,C57BL/6J,None,3,None,2019-08-06 21:30:42


In [3]:
key = {'subject_uuid': UUID('05e97eea-64e2-4e52-a78a-5b78875b4fa3'), 'last_session_date': datetime.date(2019, 9, 22)}
subj = subject.Subject & 'subject_nickname="KS004"'
# get trial counts and session length to date
sessions = (behavior.BehavioralSummaryByDate.PsychResults & 'prob_left=0.5' &
            key).proj('session_date', 'signed_contrasts', 'prob_choose_right')

# get date ranges and mondays
d = putils.get_date_range(subj)

# get contrast and p_prob_choose_right per day
contrast_list = []
for day in d['date_array']:
    if sessions & {'session_date': day}:
        session = (sessions & {'session_date': day}).fetch(as_dict=True)
        session = session[0]
        for icontrast, contrast in enumerate(session['signed_contrasts']):
            contrast_list.append({'session_date': session['session_date'], 
                                 'signed_contrast': round(contrast, 2)*100, 
                                 'prob_choose_right': session['prob_choose_right'][icontrast]})
    else:
        contrast_list.append({'session_date': day,
                              'signed_contrast': 100,
                              'prob_choose_right': np.nan
            
        })
contrast_df = pd.DataFrame(contrast_list)
contrast_map = contrast_df.pivot(
    'signed_contrast', 'session_date', 'prob_choose_right').sort_values(
        by='signed_contrast', ascending=False)

contrast_map = contrast_map.where(pd.notnull(contrast_map), None)
contrasts = np.sort(contrast_df['signed_contrast'].unique())

In [24]:
data = [dict(
    x=[t.strftime('%Y-%m-%d') 
       for t in contrast_map.columns.tolist()],
    y=list(range(len(contrast_map.index.tolist())))[::-1],
    z=contrast_map.values.tolist(),
    zmax=1,
    zmin=0,
    xgap=1,
    ygap=1,
    type='heatmap',
    colorbar=dict(
        thickness=10,
        title='prob choosing right',
        titleside = 'right',
    )
    
)]

data = putils.create_monday_plot(data, [-100, 100], d['mondays'])

layout = go.Layout(
    xaxis=dict(
        title='Date',
        showgrid=False
    ),
    yaxis=dict(
        title='Contrast (%)',
        range=[0, len(contrast_map.index.tolist())],
        tickmode = 'array',
        tickvals = list(range(0, len(contrast_map.index.tolist()))),
        ticktext = [str(contrast) for contrast in contrasts]
    ),
    width=700,
    height=400,
    title=dict(
        text='Contrast heatmap',
        x=0.3,
        y=0.85
    ),
    legend=dict(
        x=1.2,
        y=0.8,
        orientation='v'
    ),
    template=dict(
        layout=dict(
           plot_bgcolor="white"
        )
    )
    
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [11]:
f = open("contrast_heatmap.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()